In [101]:
import tarfile
import pickle
import sagemaker
import pandas as pd

SEE HOW TO IMPORT parameterdict FROM PICKLED FILE WITHOUT HAVING TO REDIFINE THE parameterdict CLASS

In [113]:
#AIM IS TO REMOVE THIS CELL
class parameterdict(dict):
    def __init__(self, file_name):
        self.file_name = file_name
        
    def store(self):
        with open(self.file_name, 'ab') as dbfile:
            pickle.dump(self, dbfile)                      
        return
    
    def load(self):  
        with open(self.file_name, 'rb') as dbfile:     
            db = pickle.load(dbfile)  
        return db

In [114]:
with open('titanic_parameters.pkl', 'rb') as dbfile:
    PARAMETERS = pickle.load(dbfile)

In [115]:
PARAMETERS

{'my_region': 'eu-west-1',
 'bucket_name': 's3.sagemaker.end2endtitanic.7ededbe5308646a3adbc'}

In the 'train' notebook we trained an XGBoost model, now by using the 'attach' function we can attach the model artifacts in order to instanciate a new XGBoost model that we can deploy

In [97]:
xgb_model = sagemaker.estimator.Estimator.attach('xgboost-2019-07-15-18-56-15-081')

2019-07-15 18:59:08 Starting - Preparing the instances for training
2019-07-15 18:59:08 Downloading - Downloading input data
2019-07-15 18:59:08 Training - Training image download completed. Training in progress.
2019-07-15 18:59:08 Uploading - Uploading generated training model
2019-07-15 18:59:08 Completed - Training job completedArguments: train
[2019-07-15:18:58:57:INFO] Running standalone xgboost training.
[2019-07-15:18:58:57:INFO] File size need to be processed in the node: 0.02mb. Available memory size in the node: 8464.18mb
[2019-07-15:18:58:57:INFO] Determined delimiter of CSV input is ','
[18:58:57] S3DistributionType set as FullyReplicated
[18:58:57] 668x7 matrix with 4676 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2019-07-15:18:58:57:INFO] Determined delimiter of CSV input is ','
[18:58:57] S3DistributionType set as FullyReplicated
[18:58:57] 223x7 matrix with 1561 entries loaded from /opt/ml/input/data/validation?format=csv&label_c

To test that everything we did so far makes sense we have create a batch transform that makes predictions on our test data. This will then be uploaded to Kaggle

In [99]:
batch_input = 's3://s3.sagemaker.end2endtitanic.7ededbe5308646a3adbc/data/for_xgboost/test.csv' # The location of the test dataset

# The location to store the results of the batch transform job
batch_output = f"s3://{PARAMETERS['bucket_name']}/batch-inference" 

transformer = xgb_model.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path=batch_output)

transformer.transform(data=batch_input, data_type='S3Prefix', content_type='text/csv', split_type='Line')

transformer.wait()

.......................................!


Putting the predictions in format required by Kaggle

In [141]:
test_data = pd.read_csv( 's3://s3.sagemaker.end2endtitanic.7ededbe5308646a3adbc/data/raw/test' ) #PARAMETERS['raw_test_data_path'])
batch_prediction = pd.read_csv('s3://s3.sagemaker.end2endtitanic.7ededbe5308646a3adbc/batch-inference/test.csv.out',names=['Survived'])
output = pd.DataFrame( {'PassengerId': test_data.PassengerId ,'Survived': (batch_prediction.Survived>=0.5) + 0})
output.to_csv( 'titanic_predictions.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


We got an acuracy of 0.76076 without putting any thought into the actual model. The only data processing step we performed was label encoding the gender and the point of embarkation.	